# 🧱 Tokenization

Welcome to this notebook on **tokenization**, the essential first step that turns raw text into something a model can actually work with. Before any language model can “understand” text, the text must be broken into tokens: characters, subwords, or other meaningful units. How we choose to do that has a huge impact on model performance, vocabulary size, efficiency, and even how well the model handles rare or creative language.

In this notebook, we’ll explore tokenization from the ground up:

- 🔤 Starting with raw bytes  
- 🧩 Building up to subword methods like **Byte Pair Encoding (BPE)**  
- ⚠️ Examining common pitfalls and weird edge cases  
- 🔍 Investigating how different snippets of text get tokenized in practice  

Along the way, you’ll experiment, compare techniques, and uncover the surprising ways models “see” text. By the end, you’ll not only understand how modern tokenizers work, you’ll be able to build and analyze your own.

Let’s dive in and start breaking things apart! ✨

In [ ]:
import tiktoken
from typing import List, Tuple

## 🌟 Byte Pair Encoding (BPE)

You’ve already seen that tokenization is about breaking text into pieces a model can understand. But language is huge and unpredictable: new words, names, typos, different spellings. If we tried to create a token for *every* word, our vocabulary would be enormous… and still fail on words we haven’t seen! 😅

**Byte Pair Encoding (BPE)** offers a clever fix. Instead of learning whole words, BPE learns **subword units**, ie. frequent chunks that can be combined to form any word. This gives us:

- 🔤 **Complete coverage:** Even brand-new words can be built from familiar subwords.  
- 📦 **Smaller vocabulary:** Less memory, faster models, cleaner representation.

### 🧠 The big idea  
Start with characters → repeatedly merge the most common pair → build bigger and bigger chunks. Over time, the tokenizer “discovers” useful patterns like roots, prefixes, and common letter combos.

There are different ways to implement this algorithm. In this section, we will build together, a very simple version of it just for the purpose of understanding how it works and seeing it in action.

Ready? Let’s start merging! 🔗✨


Let’s start by loading a few early scenes from Monty Python and the Holy Grail, stored in `monty_python.txt`. Then to make things easy, let's encode all characters using UTF-8 and save them in a list, representing each byte with an integer.

**`TODO:`** Load `monty_python.txt` as a `str`. Then encode each character using UTF-8 and in a list save the byte representing each character as an `int`.

In [ ]:
with open("monty_python.txt", "r") as file:
    text = file.read()

tokens = ...

**`Discussion:`** Right now we have already encoded all characters of our initial text. Is this enough? If not, please provide the reasons why.

\[Your Answer\]

In the next step we need to be able to compute the number of times a unique pair of characters (or in our case ids), appear in the corpus. To do this, we simply need to iterate through all pairs in our corpus and count it.

**`TODO:`** Complete the following function. The function receives a `list` of integers (our tokens) and counts the number of times each pair of consecutive integers appears in the list in a `dict`.

In [ ]:
def get_stats(ids: List[int]) -> dict:
    """
    Get counts of all adjacent token pairs in the list of token IDs.
    Args:
        ids (List[int]): List of token IDs.
    Returns:
        dict: A dictionary with token pairs as keys and their counts as values.
    """

In the next stage, we have to implement the merging step of the algorithm. In this step, we have already identified the most frequent pair (`pair`) and we'd like to replace it with a new representation `idx` that does not currently exist in our vocabulary.

**`TODO:`** Complete the following function. The function receives the current representation of our tokens (`ids`), the most frequent pair (`pair`) and the new token with which we'd like to replace the most frequent pair (`idx`). Using this information it updates our token representation by replacing the most frequent pair with our indicated new token.

In [ ]:
def merge(ids: List[int], pair: Tuple[int, int], idx: int) -> List[int]:
  """
  Merge all occurrences of the specified token pair in the list of token IDs.
  Args:
    ids (List[int]): List of token IDs.
    pair (Tuple[int, int]): The token pair to merge.
    idx (int): The new token ID to replace the pair.
  Returns:
    List[int]: The updated list of token IDs with the pair merged.
  """

Now that we have all the components that we require let's put them in a loop and see if our algorithm works 😁

In [ ]:
vocab_size = 276 # The desired final vocabulary size
num_merges = vocab_size - 256 # The number of merges to perform (from initial 256 byte tokens)
ids = list(tokens) # copy so we don't destroy the original list

merges = {}
for i in range(num_merges):
  stats = get_stats(ids)
  pair = max(stats, key=stats.get)
  idx = 256 + i
  print(f"merging {pair} into a new token {idx}")
  ids = merge(ids, pair, idx)
  merges[pair] = idx

print()
print(f"Initial token count: {len(tokens)}")
print(f"Final token count: {len(ids)}")

## ⚠️ Common Problems in Tokenization

Tokenization may *look* simple… but in practice, it’s full of little traps and surprises. 😬  
Different languages, punctuation quirks, emojis, hyphens, typos, and even whitespace can trip up a tokenizer or lead to awkward, inefficient splits.

In this section, we’ll explore some of the most common issues you’ll face when turning raw text into tokens, including:

- 🌀 **Inconsistent splitting**
- 🌍 **Multilingual and Unicode challenges**  
- 😵 **Strange artifacts from messy text**  
- 🔗 **How subword methods sometimes break words in weird places**

We’ll look at examples, and think about how modern tokenizers deal with these challenges.  
Ready to dive into the chaos? Let’s debug some tokens! 🛠️✨


### ⚡ `Tiktoken`

To experiment with modern tokenization, we’ll use **tiktoken**, a lightweight and fast tokenizer library designed for models like GPT. It’s built for **speed**, **efficiency**, and **consistency** with real production tokenizers.

With it, you can:

- 🔢 Encode text into tokens (and decode back)  
- 📏 Count tokens quickly  
- 🚀 Try out different tokenization schemes used by current language models  

We’ll use it for the rest of this notebook to see how real LLM tokenizers behave. Let's have a look at a quick example.


In [ ]:
text = "Hey everyone! Good luck with your Project Milestone P3 submissions!"

encoder = tiktoken.get_encoding("gpt2")
tokens = encoder.encode(text)

print(text)
print(tokens)
print("Token count:", len(tokens))


### 🔍 Investigating Tokenization Examples

In this part, you’ll investigate how different pieces of text are tokenized.  
Your goal is simply to **look closely**, notice patterns, think about *why* the tokenizer behaves the way it does, and how this could be a problem. 🕵️‍♂️✨

You can explore the examples in **any way you prefer**:

- ▶️ **Run your own code** using `tiktoken`, or  
- 🌐 Use the online tokenizer tool: [link](https://tiktokenizer.vercel.app)  

If you choose the web app, make sure to select the **gpt2** tokenizer so that your results match ours.

Take your time, experiment, and follow your curiosity—there are lots of interesting surprises hiding in how text gets chopped into tokens!


**`Discussion:`** Experiments with the examples below. Tokenize them and inspect how the resulting tokens are formed. Can you see any concerning / unexpected outcomes?

In [ ]:
example1 = "127 + 677 = 804"
example2 = "1275 + 6773 = 8041"

\[Your Answer\]

**`Discussion:`** Experiments with the examples below. Tokenize them and inspect how the resulting tokens are formed. Can you see any concerning / unexpected outcomes?

In [ ]:
example1 = "Egg."
example2 = "I have an Egg."
example3 = "egg."
example4 = "EGG." 

\[Your Answer\]

**`Discussion:`** Experiments with the examples below. Tokenize them and inspect how the resulting tokens are formed. Can you see any concerning / unexpected outcomes?

- *Note:* Examples `example2` and `example3` are french and chinese translations of `example1`

In [ ]:
example1 = """Once when I was six years old I saw a magnificent picture in a book, called True Stories from Nature,about the primeval forest. It was a picture of a boa constrictor in the act of swallowing an animal."""
example2 = """Lorsque j’avais six ans j’ai vu, une fois, une magnifique image, dans un livre sur la Forêt Vierge qui s’appelait « Histoires Vécues ». Ça représentait un serpent boa qui avalait un fauve.""" 
example3 = """我六岁的时候，在一本叫做《来自大自然的真实故事》的书里，看到一张非常壮观的图片，书里讲的是原始森林。图片上是一条蟒蛇正在吞食一只动物。"""

for example in [example1, example2, example3]:
    print(f"\nExample: {example}")
    tokens = encoder.encode(example)
    print(f"Number of tokens: {len(tokens)}")

\[Your Answer\]

**`Discussion:`** Experiments with the examples below. Tokenize them and inspect how the resulting tokens are formed. Can you see any concerning / unexpected outcomes?

In [ ]:
example1 = """
for i in range(1, 101):
    if i % 3 == 0 and i % 5 == 0:
        print("FizzBuzz")
"""

tokens = encoder.encode(example1)
for token in tokens:
    print(f"'{encoder.decode([token])}' -> {token}")

\[Your Answer\]

In [ ]:
example1 = """
for i in range(1, 101):
    if i % 3 == 0 and i % 5 == 0:
        print("FizzBuzz")
"""

encoder_gpt4 = tiktoken.get_encoding("cl100k_base")
tokens = encoder_gpt4.encode(example1)
for token in tokens:
    print(f"'{encoder_gpt4.decode([token])}' -> {token}")